# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import pandas as pd
import psycopg2
from sql_queries import airport_insert, demographic_insert, immigration_insert, temperature_insert

Step 1 : Scope the Project and Gather Data

Scope
This project aims to enrich the US I94 immigration data with further data such as dempgraphics, temperature and airport data 
to have a wider basis for analysis on the immigration data.

I94 Immigration Data:
This data comes from the US National Tourism and Trade Office. A data dictionary is included in the workspace.
This is https://travel.trade.gov/research/reports/i94/historical/2016.html where the data comes from. There's a
sample file so you can take a look at the data in csv format before reading it all in. You do not have to use
the entire dataset, just use what you need to accomplish the goal you set at the beginning of the project.m

In [2]:
path = '../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat'
df_i94 = pd.read_sas(path, 'sas7bdat', encoding='ISO-8859-1')

In [3]:
df_i94.columns

Index(['cicid', 'i94yr', 'i94mon', 'i94cit', 'i94res', 'i94port', 'arrdate',
       'i94mode', 'i94addr', 'depdate', 'i94bir', 'i94visa', 'count',
       'dtadfile', 'visapost', 'occup', 'entdepa', 'entdepd', 'entdepu',
       'matflag', 'biryear', 'dtaddto', 'gender', 'insnum', 'airline',
       'admnum', 'fltno', 'visatype'],
      dtype='object')

In [4]:
df_i94.head(10)

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,6.0,2016.0,4.0,692.0,692.0,XXX,20573.0,NaN,NaN,NaN,...,U,NaN,1979.0,10282016,NaN,NaN,NaN,1.897628e+09,NaN,B2
1,7.0,2016.0,4.0,254.0,276.0,ATL,20551.0,1.0,AL,NaN,...,Y,NaN,1991.0,D/S,M,NaN,NaN,3.736796e+09,00296,F1
2,15.0,2016.0,4.0,101.0,101.0,WAS,20545.0,1.0,MI,20691.0,...,NaN,M,1961.0,09302016,M,NaN,OS,6.666432e+08,93,B2
3,16.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,NaN,M,1988.0,09302016,NaN,NaN,AA,9.246846e+10,00199,B2
4,17.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,NaN,M,2012.0,09302016,NaN,NaN,AA,9.246846e+10,00199,B2
5,18.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MI,20555.0,...,NaN,M,1959.0,09302016,NaN,NaN,AZ,9.247104e+10,00602,B1
6,19.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,NJ,20558.0,...,NaN,M,1953.0,09302016,NaN,NaN,AZ,9.247140e+10,00602,B2
7,20.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,NJ,20558.0,...,NaN,M,1959.0,09302016,NaN,NaN,AZ,9.247161e+10,00602,B2
8,21.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,NY,20553.0,...,NaN,M,1970.0,09302016,NaN,NaN,AZ,9.247080e+10,00602,B2
9,22.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,NY,20562.0,...,NaN,M,1968.0,09302016,NaN,NaN,AZ,9.247849e+10,00608,B1


World Temperature Data: This dataset came from Kaggle. You can read more about it
here https://www.kaggle.com/berkeleyearth/climate-change-earth-surface-temperature-data.

In [5]:
path = '../../data2/GlobalLandTemperaturesByCity.csv'
df_temp = pd.read_csv(path)

In [6]:
df_temp.head(10)

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Århus,Denmark,57.05N,10.33E
1,1743-12-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
2,1744-01-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
3,1744-02-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
4,1744-03-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
5,1744-04-01,5.788,3.624,Århus,Denmark,57.05N,10.33E
6,1744-05-01,10.644,1.283,Århus,Denmark,57.05N,10.33E
7,1744-06-01,14.051,1.347,Århus,Denmark,57.05N,10.33E
8,1744-07-01,16.082,1.396,Århus,Denmark,57.05N,10.33E
9,1744-08-01,NaN,NaN,Århus,Denmark,57.05N,10.33E


In [7]:
df_temp_us = df_temp[df_temp['Country']== 'United States']
df_temp_us.head(10)

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
47555,1820-01-01,2.101,3.217,Abilene,United States,32.95N,100.53W
47556,1820-02-01,6.926,2.853,Abilene,United States,32.95N,100.53W
47557,1820-03-01,10.767,2.395,Abilene,United States,32.95N,100.53W
47558,1820-04-01,17.989,2.202,Abilene,United States,32.95N,100.53W
47559,1820-05-01,21.809,2.036,Abilene,United States,32.95N,100.53W
47560,1820-06-01,25.682,2.008,Abilene,United States,32.95N,100.53W
47561,1820-07-01,26.268,1.802,Abilene,United States,32.95N,100.53W
47562,1820-08-01,25.048,1.895,Abilene,United States,32.95N,100.53W
47563,1820-09-01,22.435,2.216,Abilene,United States,32.95N,100.53W
47564,1820-10-01,15.830,2.169,Abilene,United States,32.95N,100.53W


U.S. City Demographic Data: This data comes from OpenSoft. You can read more about it
here https://public.opendatasoft.com/explore/dataset/us-cities-demographics/export/ .

In [8]:
path = 'us-cities-demographics.csv'
df_demo = pd.read_csv(path, delimiter=';')

In [9]:
df_demo.head(10)

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601.0,41862.0,82463,1562.0,30908.0,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129.0,49500.0,93629,4147.0,32935.0,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040.0,46799.0,84839,4819.0,8229.0,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127.0,87105.0,175232,5821.0,33878.0,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,White,76402
5,Peoria,Illinois,33.1,56229.0,62432.0,118661,6634.0,7517.0,2.40,IL,American Indian and Alaska Native,1343
6,Avondale,Arizona,29.1,38712.0,41971.0,80683,4815.0,8355.0,3.18,AZ,Black or African-American,11592
7,West Covina,California,39.8,51629.0,56860.0,108489,3800.0,37038.0,3.56,CA,Asian,32716
8,O'Fallon,Missouri,36.0,41762.0,43270.0,85032,5783.0,3269.0,2.77,MO,Hispanic or Latino,2583
9,High Point,North Carolina,35.5,51751.0,58077.0,109828,5204.0,16315.0,2.65,NC,Asian,11060


Airport Code Table: This is a simple table of airport codes and corresponding cities. It comes 
from here https://datahub.io/core/airport-codes#data . 

In [10]:
path = 'airport-codes_csv.csv'
df_air = pd.read_csv(path)

In [11]:
df_air.head(10)

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11.0,NaN,US,US-PA,Bensalem,00A,NaN,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435.0,NaN,US,US-KS,Leoti,00AA,NaN,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450.0,NaN,US,US-AK,Anchor Point,00AK,NaN,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820.0,NaN,US,US-AL,Harvest,00AL,NaN,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237.0,NaN,US,US-AR,Newport,NaN,NaN,NaN,"-91.254898, 35.6087"
5,00AS,small_airport,Fulton Airport,1100.0,NaN,US,US-OK,Alex,00AS,NaN,00AS,"-97.8180194, 34.9428028"
6,00AZ,small_airport,Cordes Airport,3810.0,NaN,US,US-AZ,Cordes,00AZ,NaN,00AZ,"-112.16500091552734, 34.305599212646484"
7,00CA,small_airport,Goldstone /Gts/ Airport,3038.0,NaN,US,US-CA,Barstow,00CA,NaN,00CA,"-116.888000488, 35.350498199499995"
8,00CL,small_airport,Williams Ag Airport,87.0,NaN,US,US-CA,Biggs,00CL,NaN,00CL,"-121.763427, 39.427188"
9,00CN,heliport,Kitchen Creek Helibase Heliport,3350.0,NaN,US,US-CA,Pine Valley,00CN,NaN,00CN,"-116.4597417, 32.7273736"


Step 2: Explore and Assess the Data

Explore the data
Identify data quality issues, like missing values and duplicates data etc.

In [12]:
df_i94.describe()

,cicid,i94yr,i94mon,i94cit,i94res,arrdate,i94mode,depdate,i94bir,i94visa,count,biryear,admnum
count,3.096313e+06,3096313.0,3096313.0,3.096313e+06,3.096313e+06,3.096313e+06,3.096074e+06,2.953856e+06,3.095511e+06,3.096313e+06,3096313.0,3.095511e+06,3.096313e+06
mean,3.078652e+06,2016.0,4.0,3.049069e+02,3.032838e+02,2.055985e+04,1.073690e+00,2.057395e+04,4.176761e+01,1.845393e+00,1.0,1.974232e+03,7.082885e+10
std,1.763278e+06,0.0,0.0,2.100269e+02,2.085832e+02,8.777339e+00,5.158963e-01,2.935697e+01,1.742026e+01,3.983910e-01,0.0,1.742026e+01,2.215442e+10
min,6.000000e+00,2016.0,4.0,1.010000e+02,1.010000e+02,2.054500e+04,1.000000e+00,1.517600e+04,-3.000000e+00,1.000000e+00,1.0,1.902000e+03,0.000000e+00
25%,1.577790e+06,2016.0,4.0,1.350000e+02,1.310000e+02,2.055200e+04,1.000000e+00,2.056100e+04,3.000000e+01,2.000000e+00,1.0,1.962000e+03,5.603523e+10
50%,3.103507e+06,2016.0,4.0,2.130000e+02,2.130000e+02,2.056000e+04,1.000000e+00,2.057000e+04,4.100000e+01,2.000000e+00,1.0,1.975000e+03,5.936094e+10
75%,4.654341e+06,2016.0,4.0,5.120000e+02,5.040000e+02,2.056700e+04,1.000000e+00,2.057900e+04,5.400000e+01,2.000000e+00,1.0,1.986000e+03,9.350987e+10
max,6.102785e+06,2016.0,4.0,9.990000e+02,7.600000e+02,2.057400e+04,9.000000e+00,4.542700e+04,1.140000e+02,3.000000e+00,1.0,2.019000e+03,9.991557e+10


In [13]:
df_temp_us.describe()

,AverageTemperature,AverageTemperatureUncertainty
count,661524.000000,661524.00000
mean,13.949335,1.08955
std,9.173337,1.15068
min,-25.163000,0.04000
25%,7.787000,0.30000
50%,14.922000,0.52400
75%,21.081000,1.64600
max,34.379000,10.51900


In [14]:
df_demo.describe()

,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,Count
count,2891.000000,2.888000e+03,2.888000e+03,2.891000e+03,2878.000000,2.878000e+03,2875.000000,2.891000e+03
mean,35.494881,9.732843e+04,1.017696e+05,1.989668e+05,9367.832523,4.065360e+04,2.742543,4.896377e+04
std,4.401617,2.162999e+05,2.315646e+05,4.475559e+05,13211.219924,1.557491e+05,0.433291,1.443856e+05
min,22.900000,2.928100e+04,2.734800e+04,6.321500e+04,416.000000,8.610000e+02,2.000000,9.800000e+01
25%,32.800000,3.928900e+04,4.122700e+04,8.042900e+04,3739.000000,9.224000e+03,2.430000,3.435000e+03
50%,35.300000,5.234100e+04,5.380900e+04,1.067820e+05,5397.000000,1.882200e+04,2.650000,1.378000e+04
75%,38.000000,8.664175e+04,8.960400e+04,1.752320e+05,9368.000000,3.397175e+04,2.950000,5.444700e+04
max,70.500000,4.081698e+06,4.468707e+06,8.550405e+06,156961.000000,3.212500e+06,4.980000,3.835726e+06


In [15]:
df_air.describe()

,elevation_ft
count,48069.000000
mean,1240.789677
std,1602.363459
min,-1266.000000
25%,205.000000
50%,718.000000
75%,1497.000000
max,22000.000000


Cleaning Steps 
Document steps necessary to clean the data

In [16]:
df_i94.isna().sum().sum()

12139654

In [17]:
# Get port locations from SAS text file
with open("./I94_SAS_Labels_Descriptions.SAS") as f:
    content = f.readlines()
content = [x.strip() for x in content]
#print(f"content is {content}")
ports = content[302:962]
#print(f"Port is {ports}")
splitted_ports = [port.split("=") for port in ports]
port_codes = [x[0].replace("'","").strip() for x in splitted_ports]
#print(f"port_codes is {port_codes}")
port_locations = [x[1].replace("'","").strip() for x in splitted_ports]
#print(f"port-locations is {port_locations}")
port_cities = [x.split(",")[0] for x in port_locations]
port_states = [x.split(",")[-1] for x in port_locations]
df_port_locations = pd.DataFrame({"port_code" : port_codes, "port_city": port_cities, "port_state": port_states})
df_port_locations.head(10)

,port_code,port_city,port_state
0,ALC,ALCAN,AK
1,ANC,ANCHORAGE,AK
2,BAR,BAKER AAF - BAKER ISLAND,AK
3,DAC,DALTONS CACHE,AK
4,PIZ,DEW STATION PT LAY DEW,AK
5,DTH,DUTCH HARBOR,AK
6,EGL,EAGLE,AK
7,FRB,FAIRBANKS,AK
8,HOM,HOMER,AK
9,HYD,HYDER,AK


In [18]:
df_port_locations.count()

port_code     660
port_city     660
port_state    660
dtype: int64

In [19]:
df_port_locations.isna().sum()

port_code     0
port_city     0
port_state    0
dtype: int64

In [20]:
df_port_locations[df_port_locations['port_code']=='OSN']

,port_code,port_city,port_state
659,OSN,No PORT Code (OSN),No PORT Code (OSN)


In [21]:
irregular_ports_df = df_port_locations[df_port_locations["port_city"] == df_port_locations["port_state"]]

In [22]:
irregular_ports_df.count()

port_code     77
port_city     77
port_state    77
dtype: int64

In [23]:
irregular_ports_df.head(10)

,port_code,port_city,port_state
28,MAP,MARIPOSA AZ,MARIPOSA AZ
76,WAS,WASHINGTON DC,WASHINGTON DC
516,XXX,NOT REPORTED/UNKNOWN,NOT REPORTED/UNKNOWN
517,888,UNIDENTIFED AIR / SEAPORT,UNIDENTIFED AIR / SEAPORT
518,UNK,UNKNOWN POE,UNKNOWN POE
575,ZZZ,MEXICO Land (Banco de Mexico),MEXICO Land (Banco de Mexico)
576,CHN,No PORT Code (CHN),No PORT Code (CHN)
578,MAA,Abu Dhabi,Abu Dhabi
591,FRG,Collapsed (FOK) 06/15,Collapsed (FOK) 06/15
592,HRL,Collapsed (HLG) 06/15,Collapsed (HLG) 06/15


In [24]:
irregular_ports = list(set(irregular_ports_df["port_code"].values))
print(irregular_ports)

['JFA', 'ASI', 'JIG', 'JMZ', 'NC8', 'AUH', 'PCW', 'NYL', 'MAP', 'FRG', 'XXX', 'ATW', 'HRL', 'GAC', 'WA5', 'IAG', 'JSJ', 'DAY', '5T6', 'ADU', 'MTH', 'AKT', 'ISP', 'OAI', 'SP0', 'CPX', 'Y62', 'PHN', 'TIW', 'JBQ', 'GPI', '.GA', 'NK', 'DEC', 'OSN', 'T01', 'FTB', 'WTR', 'W55', 'BUS', 'FSC', 'SCH', 'WAS', 'RYY', 'CHN', 'EGE', 'CXO', 'BCM', 'VMB', 'XNA', '060', 'CLX', 'X96', 'CP', 'A2A', 'BKF', 'OLM', 'AMT', 'NGL', 'YGF', 'X44', 'LIT', 'OGS', 'AG', 'OTS', 'SUS', 'PFN', 'ZZZ', 'STN', 'UNK', '74S', 'PLB', 'DRV', 'GMT', 'PHF', 'MAA', '888']


In [25]:
r, c = df_i94.shape

In [26]:
pd.options.mode.chained_assignment = None 

In [27]:
print(f"I94 row count is {r} before cleaning the data")
df_i94_filtered = df_i94[~df_i94["i94port"].isin(irregular_ports)]
print(f"i94 data contains {len(df_i94_filtered)} rows after removing irregular ports.ik")
df_i94_filtered.drop(columns=["insnum", "entdepu", "occup", "visapost"], inplace=True)
df_i94_filtered.dropna(inplace=True)
print(f"i94 data contains {len(df_i94_filtered)} rows after removing NaN values.")

I94 row count is 3096313 before cleaning the data
i94 data contains 2995590 rows after removing irregular ports.ik
i94 data contains 2306754 rows after removing NaN values.


In [28]:
print(f"df_temp_us row count is {len(df_temp_us)} before removing Nan")
df_temp_us.dropna(inplace=True)
print(f"df_temp_us row count is {len(df_temp_us)} after removing Nan")

df_temp_us row count is 687289 before removing Nan
df_temp_us row count is 661524 after removing Nan


In [29]:
print(f"df_air row count is {len(df_air)} before removing Nan")
df_air.dropna(subset=['iata_code'],inplace=True)
print(f"df_air row count is {len(df_air)} after removing Nan")

df_air row count is 55075 before removing Nan
df_air row count is 9189 after removing Nan


In [30]:
print(f"df_demo row count is {len(df_demo)} before removing Nan")
df_demo.dropna(inplace=True)
print(f"df_demo row count is {len(df_demo)} after removing Nan")

df_demo row count is 2891 before removing Nan
df_demo row count is 2875 after removing Nan


In [32]:
df_i94_filtered.shape

(2306754, 24)

In [ ]:
df_i94_filtered=df_i94_filtered.replace(' ', np.nan, regex=True)

In [47]:
df_i94_filtered.isna().sum()

cicid       0
i94yr       0
i94mon      0
i94cit      0
i94res      0
i94port     0
arrdate     0
i94mode     0
i94addr     0
depdate     0
i94bir      0
i94visa     0
count       0
dtadfile    0
entdepa     0
entdepd     0
matflag     0
biryear     0
dtaddto     1
gender      0
airline     0
admnum      0
fltno       1
visatype    0
dtype: int64

In [ ]:
df_i94_filtered.dropna(inplace=True)

In [49]:
df_i94_filtered.shape

(2306752, 24)

Step 3 :Define the Data Model

3.1 Conceptual Data Model
Map out the conceptual data model and explain why you have chosen that model.

Tables:

|table name||columns||description||type|
|---||---||---||---|
|airports||iata_code-name-type-local_code-coordinates-city||stores information related to airports||dimension table|
|demographics||city-data-media_age-male_population-female-population-total_population-num_veterans-foreign_born-average_household_size-state_code-race-count||stores demographics data for cities||dimension table|
|immigrations||cicid-year-month-cit-res-iata-arrdate-mode-addr-depdate-bir-visa-coun-dtadfil-visapost-occup-entdepa-entdepd-matflag-biryear-dtaddto-gender-insnum-airline-admnum-fitno-visatype||stores all i94 immigrations data||fact table|
|temperature||timestamp-average_temperature-average_temperature_uncertanity-city-country-latitude-longitude||stores temperature information||dimension table|

3.2 Mapping out Data pipelines

Below are the List of steps required to pipeline the data into the chosen data Model-
1. Create tables by excecuting create_tables.py
2. Join city to airports data
3. Insert data.

Step 4 : Run Pipelines to Model the Data
    
4.1 Create the data model
Build the data piplines to create the data model

In [50]:
!python create_tables.py

In [51]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [92]:
df_air = df_air.merge(df_port_locations, left_on="iata_code", right_on="port_code")
df_air.head(5)

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates,port_code,port_city,port_state
0,57A,seaplane_base,Tokeen Seaplane Base,NaN,NaN,US,US-AK,Tokeen,57A,TKI,57A,"-133.32699585, 55.9370994568",TKI,TOKEEN,AK
1,5Z9,seaplane_base,Lake Brooks Seaplane Base,36.0,NaN,US,US-AK,Katmai National Park,5Z9,BKF,5Z9,"-155.77699279785, 58.554798126221",BKF,No PORT Code (BKF),No PORT Code (BKF)
2,89NY,small_airport,Maxson Airfield,340.0,NaN,US,US-NY,Alexandria Bay,89NY,AXB,89NY,"-75.90034, 44.312002",AXB,ALEXANDRIA BAY,NY
3,AGGF,small_airport,Fera/Maringe Airport,NaN,OC,SB,SB-IS,Fera Island,AGGF,FRE,NaN,"159.576996, -8.1075",FRE,FRESNO,CA
4,ANZ,small_airport,Angus Downs Airport,1724.0,OC,AU,AU-NT,Angus Downs Station,NaN,ANZ,NaN,"132.2748, -25.0325",ANZ,ANZALDUAS,TX


In [93]:
df_air.drop(columns=["port_code"], inplace=True)

In [94]:
df_air = df_air[["iata_code", "name", "type", "local_code", "coordinates", "port_city", "elevation_ft", "continent", \
               "iso_country", "iso_region", "municipality", "gps_code"]]


In [55]:
df_air.columns

Index(['iata_code', 'name', 'type', 'local_code', 'coordinates', 'port_city',
       'elevation_ft', 'continent', 'iso_country', 'iso_region',
       'municipality', 'gps_code'],
      dtype='object')

In [75]:
df_air = df_air.drop_duplicates('iata_code', keep='first')
# mask = df_air['iata_code'].value_counts()[df_air['iata_code'].value_counts().sort_values(ascending=False) > 1]
# df_air[df_air['iata_code'].isin(mask.index)]

In [108]:
for index, row in df_air.iterrows():
    cur.execute(airport_insert, list(row.values))
    conn.commit()

In [72]:
for index, row in df_demo.iterrows():
    cur.execute(demographic_insert, list(row.values))
    conn.commit()

In [53]:
from tqdm import tqdm 

In [ ]:
for index, row in tqdm(df_i94_filtered.iterrows(), total=len(df_i94_filtered)):
    cur.execute(immigration_insert, list(row.values))
    conn.commit()

In [ ]:
for index, row in df_temp_us.iterrows():
    cur.execute(temperature_insert, list(row.values))
    conn.commit()

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here
cur.execute("SELECT COUNT(*) FROM airports")
conn.commit()
if cur.rowcount < 1:
    print("No data found in table airports")
    
cur.execute("SELECT COUNT(*) FROM demographics")
conn.commit()
if cur.rowcount < 1:
    print("No data found in table demographics")
    
cur.execute("SELECT COUNT(*) FROM immigrations")
conn.commit()
if cur.rowcount < 1:
    print("No data found in table immigrations")
    
cur.execute("SELECT COUNT(*) FROM temperature")
conn.commit()
if cur.rowcount < 1:
    print("No data found in table temperature")

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

Step 5: Complete Project Write Up
Clearly state the rationale for the choice of tools and technologies for the project.
Propose how often the data should be updated and why.
Write a description of how you would approach the problem differently under the following scenarios:
1. The data was increased by 100x.
--> Would have used pyspark so that the data efficiently get distributed and hence the processing of
    data performance get increased. For eg. EMR
2. The data populates a dashboard that must be updated on a daily basis by 7am every day.
--> In this case i will use airflow and create DAG that performs the logic of the described pipeline automaticaly.
3. The database needed to be accessed by 100+ people.
--> Use Redshift to have the data store in a way that it can efficiently be accessed by many people.

In [57]:
conn.close()